# Section 5-6: Use the sampler/estimator primitive

## TASK 5.1/6.1: Set sampler/estimator primitive options

1. Which snippet correctly transpiles a parameterized circuit for a chosen backend and produces an ISA circuit before using Estimator V2?

a. 
```python
from qiskit.transpiler import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circ = pm.run(circ)
```
b. 
```python
isa_circ = circ  # QPUs accept any gates; transpilation is optional
```
c. 
```python
from qiskit import transpile
isa_circ = transpile(circ)  # omit backend
```
d. 
```python
pm = generate_preset_pass_manager(backend=None)
isa_circ = pm.run(circ.decompose())
```


:::{dropdown}answer 
The answer is a. 

You must generate a preset pass manager with a concrete `backend` and run it to obtain an ISA circuit; this matches the example that uses `generate_preset_pass_manager(optimization_level=1, backend=backend)` followed by `pm.run(circuit)`. This step ensures the circuit uses only instructions supported by the target device prior to invoking primitives.
:::


2. You need to configure the Sampler to use 4096 shots by default for all runs in a session. Which method should you use?

a. `sampler.options.default_shots = 4096`<br>
b. `sampler.set_shots(4096)`<br>
c. `sampler.options.shots.default = 4096`<br>
d. `sampler.run(..., shots=4096)`<br>

:::{dropdown}answer 
The answer is a.

`Sampler` exposes a top‑level option `default_shots` that sets the default number of repetitions used when circuits are executed. Setting `sampler.options.default_shots = 4096` applies to subsequent `run` calls unless overridden. The other choices either refer to non‑existent attributes (`set_shots`, `options.shots.default`) or set shots per‑call rather than as the default for the options object.
:::


3. Which call correctly submits work to Estimator V2 using the Primitive Unified Bloc (PUB) tuple?

a. 
```python
job = estimator.run([(isa_circ, isa_obs, param_values)])
```
b. 
```python
job = estimator.run([(isa_obs, isa_circ, param_values)])
```
c. 
```python
job = estimator.run([(isa_circ, param_values)])
```
d. 
```python
job = estimator.run((isa_circ, isa_obs, param_values))
```


:::{dropdown}answer 
The answer is a. 

The Estimator expects a sequence of PUB tuples in the form `(circuit, observable, parameter_values)`; the example shows `estimator.run([(isa_circuit, isa_observable, param_values)])`.
:::


4. For Sampler V2, which preparation is correct before transpilation and submission?

a. 
```python
circ.measure_all()
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circ = pm.run(circ)
job = sampler.run([(isa_circ, param_values)])
```
b. 
```python
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa_circ = pm.run(circ)  # no measurements needed
job = sampler.run([(isa_circ,)])
```
c. 
```python
job = sampler.run([(circ,)])
```
d. 
```python
isa_circ = circ.remove_final_measurements(inplace=True)
job = sampler.run([(isa_circ,)])
```


:::{dropdown}answer 
The answer is a. 

The sampler example measures the circuit (`measure_all()`), transpiles to ISA with a preset pass manager for the chosen backend, and submits `[(isa_circuit, param_values)]` as the PUB.
:::


5. Which initialization correctly selects the execution mode consistent with the examples?

a. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode=backend)
```
b. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode="session")  # string literal
```
c. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator()  # mode is mandatory
```
d. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode="batch").run(backend)
```


:::{dropdown}answer 
The answer is a. 

When you initialize the Estimator, use the `mode` parameter to specify the mode you want it to run in. Possible values are `batch`, `session`, or `backend` objects for batch, session, and job execution mode, respectively.
:::


6. How can you request a backend that supports fractional gates per the guide?

a. 
```python
service = QiskitRuntimeService()
backend = QiskitRuntimeService(use_fractional_gates=True)
```
b. 
```python
service = QiskitRuntimeService()
backend = service.get_backend("fractional-gates")
```
c. 
```python
service = QiskitRuntimeService()
backend = service.least_busy(fractional=True)
```
d. 
```python
service = QiskitRuntimeService()
backend = service.least_busy(use_fractional_gates=True)
```


:::{dropdown}answer 
The answer is d. 

To use the newly supported [fractional gates](https://quantum.cloud.ibm.com/docs/guides/fractional-gates), set `use_fractional_gates=True` when requesting a backend from a `QiskitRuntimeService` instance. 

### What are fractional gates

Fractional gates are native hardware operations (e.g., **R\_X(θ)**, **R\_ZZ(θ)**) that allow arbitrary-angle rotations to be executed directly on supported IBM Quantum devices, without decomposing into multiple basis gates.

---

### When to use

* Circuits with many arbitrary rotations (e.g., VQE, QAOA, quantum simulation).
* To **reduce circuit depth**, execution time, and cumulative error.
* When the backend supports fractional gates and constraints (like angle ranges) are met.

---

### When not to use (and why)

* **Unsupported cases**: dynamic circuits, certain error-mitigation techniques (Pauli twirling, PEC, ZNE with PEA).
* **Restricted angles**: R\_ZZ(θ) only supports 0 < θ ≤ π/2.
* **Runtime parameters** may cause errors if angles are out of range.
* For simple circuits where decomposition overhead is negligible.
* It’s still an **experimental feature**, so stability and compatibility can change.
:::


7. Which code path best reflects a minimal, correct optimization/transpilation workflow before running primitives?

a. Create circuit → `generate_preset_pass_manager(optimization_level=1, backend)` → `pm.run(circuit)` → align observable with `apply_layout` → `Estimator.run([...])`<br>
b. Create circuit → call `Estimator.run(circuit)` and let the service infer gates automatically<br>
c. Create circuit → decompose to basis gates locally without a backend → `Estimator.run([...])`<br>
d. Create circuit → add barriers to prevent changes → `Estimator.run([...])`<br>


:::{dropdown}answer 
The answer is a. 

The documented steps show generating a preset pass manager with a specified backend, running it to obtain an ISA circuit, aligning the observable, and then submitting PUBs to the primitive.
:::


8. Which code correctly enables dynamical decoupling (DD) in Sampler V2 and selects the XpXm sequence?

a. 
```
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"
```

b. 
```
sampler.options.resilience.zne_mitigation = True
sampler.options.resilience.zne.extrapolator = "exponential"
```

c. 
```
sampler.options.twirling.enable_gates = True
sampler.options.twirling.sequence_type = "XpXm"
```

d. 
```
sampler.set_options(dynamical_decoupling=True, sequence_type="XpXm")
```


:::{dropdown}answer 
The answer is a. 

Sampler V2 exposes DD via `sampler.options.dynamical_decoupling`. Set `enable=True` and choose a valid `sequence_type` such as `"XpXm"`. Error mitigation like ZNE belongs to the Estimator, not Sampler. 

### Dynamic Decoupling
Dynamical decoupling works by inserting pulse sequences on idling qubits to approximately cancel out the effect of these errors. Each inserted pulse sequence amounts to an identity operation, but the physical presence of the pulses has the effect of suppressing errors. 
:::


9. You want to randomize two‑qubit gate errors via Pauli twirling when running the Sampler. Which method most directly enables this according to the twirling options API?

a. `sampler.options.twirling = {"circuit": {"enable": True}}`<br>
b. `sampler.options.twirling.enable = True`<br>
c. `sampler.options.transpilation.pauli_twirling = True`<br>
d. `sampler.enable_pauli_twirling()`<br>


:::{dropdown}answer 
The answer is b. 

Twirling controls are exposed under the `twirling` options namespace; enabling Pauli twirling is done by toggling the boolean on that namespace (e.g., `sampler.options.twirling.enable = True`) and, if needed, refining sub‑options. The other choices either use non‑documented attribute paths or nonexistent helper methods. 
:::

10. When is enabling dynamical decoupling most likely to help for Sampler runs on hardware?

a. Only when running on a simulator with noise disabled<br>
b. When all qubits are busy almost all the time (densely packed schedules)<br>
c. When circuits contain idle gaps on some qubits where no operations occur<br>
d. Only when measuring expectation values with Estimator<br>


:::{dropdown}answer 
The answer is c. 

DD is primarily beneficial when there are idle periods; pulses inserted during idle intervals can suppress coherent errors. With densely packed schedules, DD can provide little benefit or even hurt due to pulse imperfections. 
:::


11. Which error‑related options are supported directly by Sampler V2?

a. Pauli twirling (suppression) via `sampler.options.twirling.*`<br>
b. TREX measurement mitigation via `sampler.options.resilience.measure_mitigation = True`<br>
c. Zero‑noise extrapolation via `sampler.options.resilience.zne_mitigation = True`<br>
d. Probabilistic error cancellation via `sampler.options.resilience.pec_mitigation = True`<br>


:::{dropdown}answer 
The answer is a. 

Sampler supports suppression methods like Pauli twirling and dynamical decoupling. Mitigation techniques such as TREX, ZNE, and PEC are available in Estimator; Sampler does not support them directly. 
:::


12. Which snippet correctly configures Pauli twirling in Sampler V2 with 32 randomizations and 100 shots per randomization?

a. 
```
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = 32
sampler.options.twirling.shots_per_randomization = 100
```

b. 
```
sampler.options.twirling.enable = True
sampler.options.twirling.count = 32
sampler.options.twirling.shots = 100
```

c. 
```
sampler.options.resilience.twirling = True
sampler.options.resilience.num_randomizations = 32
sampler.options.resilience.shots_per_randomization = 100
```

d. 
```
sampler.enable_twirling(32, 100)
```

:::{dropdown}answer 
The answer is a. 

Use the `twirling` options on Sampler: set `enable_gates=True`, then configure `num_randomizations` and `shots_per_randomization`.
- `num_randomizations`: The number of circuit instances to draw from the ensemble of twirled circuits.
- `shots_per_randomization`: The number of shots to sample from each circuit instance.
Here’s a concise English summary of the **Concept** and **Motivation** sections of *Pauli twirling*, along with an explanation of the difference between **coherent** and **incoherent noise**:

---

### Pauli twirling

Pauli twirling is a technique where random Pauli operators (I, X, Y, Z) are inserted before and after certain gates in a quantum circuit. This process effectively transforms arbitrary noise into a *Pauli channel*, a simpler and more tractable noise model.

Quantum hardware noise often contains **coherent errors**, which accumulate *quadratically* with the number of operations, making them especially harmful. By applying Pauli twirling, these coherent errors are randomized and converted into **incoherent (Pauli) errors**, which accumulate *linearly* with circuit depth. This makes the errors easier to analyze and mitigate, and it allows error mitigation techniques that assume Pauli noise to be more effective.

---

### Coherent vs. Incoherent errors

* **Coherent errors**:

  * Systematic and unitary in nature (e.g., consistent over-rotation or phase error in a gate).
  * Error amplitudes interfere constructively over repeated operations, causing error growth to scale quadratically.
* **Incoherent errors**:

  * Probabilistic and stochastic (e.g., random bit flips, depolarizing noise).
  * Errors accumulate additively, leading to linear error growth with circuit depth.

:::


13. How do you set a repetition delay of 0.5 ms between shots for Sampler V2?

a. sampler.options.execution.rep_delay = 0.5<br>
b. sampler.options.environment.rep_delay = 0.5<br>
c. sampler.options.execution.rep_delay = 0.0005<br>
d. sampler.set_options(rep_delay_ms=0.5)<br>


:::{dropdown}answer 
The answer is c. 

`rep_delay` is specified in seconds within the `execution` options; 0.5 ms equals 0.0005 seconds. 
:::


14. When calling `SamplerV2.run`, what is the correct data type for the first positional argument?

a. A single `QuantumCircuit` instance<br>
b. A list of ISA circuits accepted by the runtime (`[isa_circuit, ...]`)<br>
c. A `Distribution` object<br>
d. A dictionary mapping bitstrings to probabilities<br>


:::{dropdown}answer 
The answer is b. 

`SamplerV2.run` expects a sequence of runtime‑compatible (ISA) circuits; passing a single `QuantumCircuit` or a probability distribution does not conform to the runtime’s `run` signature for the Sampler. (A single circuit can be provided as a one‑element list.) 
:::

15. Which statement about dynamical decoupling is TRUE?

a. DD is a measurement‑error mitigation method that post‑processes bitstrings<br>
b. Each inserted DD pulse sequence ideally implements an identity operation while suppressing coherent errors on idling qubits<br>
c. DD is guaranteed to improve results in all cases, even with dense schedules<br>
d. DD can only be used with Estimator, not with Sampler<br>


:::{dropdown}answer 
The answer is b. 

DD inserts carefully chosen pulse sequences during idle gaps. These sequences act as identity operations but help cancel coherent errors. It is a suppression (control) technique, not a post‑processing mitigation method. 
:::


16. In EstimatorV2, what does `options.default_precision` control and what is its default value?

a. It sets the target standard error for expectation values (used when shots aren’t given); default is 0.015625 (1/√4096).<br>
b. It sets the transpiler optimization level; default is 1.<br>
c. It fixes the number of shots for each circuit configuration; default is 4096.<br>
d. It controls backend run priority; default is “normal”.<br>


:::{dropdown}answer 
The answer is a. 

`default_precision` defines the default statistical precision target for expectation values when neither the PUB nor `run()` specifies precision; EstimatorV2 chooses shots to meet this. The documented default is 0.015625 (i.e., 1/√4096).
:::


17. Regarding `options.default_shots` in EstimatorV2, which statement is correct?

a. If set, it overrides `default_precision`, and when twirling is enabled it is split across randomizations.<br>
b. It is ignored whenever twirling is enabled.<br>
c. It only applies to simulators and is ignored on hardware.<br>
d. It sets the maximum number of shots per job but never overrides precision.<br>


:::{dropdown}answer 
The answer is a. 

When `default_shots` is provided, it takes precedence over `default_precision`. If Pauli twirling is active, the total shots are divided among the circuit randomizations according to the twirling options.
:::


18. Which mapping of `resilience_level` to behavior is correct for EstimatorV2?

a. 0: No mitigation; 1: Readout mitigation + measurement twirling; 2: Adds bias‑reducing estimator mitigation (medium cost).<br>
b. 0: Readout mitigation only; 1: Full error correction; 2: No mitigation.<br>
c. 0: Measurement twirling; 1: Probabilistic error cancellation; 2: Zero‑noise extrapolation only.<br>
d. 0: No mitigation; 1: Full QEC; 2: Hardware calibration reset only.<br>


:::{dropdown}answer 
The answer is a. 

`resilience_level` increases mitigation complexity: 
- 0 disables mitigation
- 1 applies minimal‑cost techniques like readout mitigation (and measurement twirling in V2)
- 2 adds medium‑cost estimator bias‑reduction methods
:::


19. For `TwirlingOptions.strategy`, what best describes the default `"active-accum"` behavior?

a. In each twirled layer, twirl the union of instruction qubits encountered up to that layer.<br>
b. Twirl only the instruction qubits of the current layer, ignoring earlier ones.<br>
c. Twirl all qubits in the circuit at every twirled layer.<br>
d. Twirl only the first two qubits of the circuit to minimize overhead.<br>


:::{dropdown}answer 
The answer is a. 

`"active-accum"` accumulates the set of instruction qubits seen so far and twirls that growing set at each subsequent twirled layer, which is default strategy.
:::


20. What is the default of `TwirlingOptions.enable_measure` and where does it apply?

a. `True` for Estimator, `False` for Sampler; enables twirling on measurement instructions not inside conditionals.<br>
b. `True` for both Estimator and Sampler; always twirl all measurements.<br>
c. `False` for both primitives; measurement twirling must be enabled via `resilience_level` only.<br>
d. `True` only on simulators; `False` on hardware backends.<br>


:::{dropdown}answer 
The answer is a. 

Measurement twirling is enabled by default for Estimator and disabled by default for Sampler. It applies to measurement operations provided they are not within a conditional block.
:::


21. When `options.resilience.zne.amplifier='pea'` is selected, which effect is correct?

a. The runtime always uses twirling and performs layer noise learning before executing amplified circuits.<br>
b. ZNE is disabled because PEA conflicts with twirling.<br>
c. Only single‑qubit gates are folded; two‑qubit gates are untouched.<br>
d. The extrapolator is forced to `'fallback'`.<br>


:::{dropdown}answer 
The answer is a. 

With PEA (Probabilistic Error Amplification), the runtime learns a twirled noise model per entangling layer (layer noise learning) and applies twirling automatically, then injects noise proportional to the learned model during execution.
:::


22. What are the documented defaults for `options.resilience.zne.noise_factors`?

a. `(1, 1.5, 2)` for PEA and `(1, 3, 5)` otherwise.<br>
b. `(1, 2, 4)` for PEA and `(1, 3, 9)` otherwise.<br>
c. Always `(1, 3, 5)` regardless of amplifier.<br>
d. Determined solely by `default_precision`.<br>


:::{dropdown}answer 
The answer is a. 

By default, PEA uses smaller incremental factors `(1, 1.5, 2)`, while gate‑folding–based amplification uses `(1, 3, 5)`.
:::


23. When ZNE options are enabled in EstimatorV2, which additional data fields are returned per PUB result?

a. `evs_extrapolated`/`stds_extrapolated` and `evs_noise_factors`/`stds_noise_factors`/`ensemble_stds_noise_factors`.<br>
b. Only `calibration_matrix` with no standard deviations.<br>
c. `zne_trace` strings and `fit_r2` only.<br>
d. No additional data; ZNE affects only `evs`/`stds`.<br>


:::{dropdown}answer 
The answer is a. 

With ZNE, in addition to `evs` and `stds`, the runtime returns extrapolated evaluations and per‑noise‑factor evaluations (including ensemble standard deviations) with shapes defined by the observable and parameter broadcasting.
:::


24. Which snippet correctly runs `SamplerV2` on two circuits and obtains quasi‑probability results?

a.
```python
from qiskit_ibm_runtime import SamplerV2
sampler = SamplerV2(mode=backend)
job = sampler.run([circ1_isa, circ2_isa])
result = job.result()
dists = result[0].data, result[1].data
```
b.
```python
from qiskit_ibm_runtime import SamplerV2
sampler = SamplerV2(mode=backend)
job = sampler.run(circ1_isa, circ2_isa)
dists = job.result().distributions
```
c.
```python
from qiskit_ibm_runtime import SamplerV2
sampler = SamplerV2(mode=backend)
result = sampler.run([circ1_isa, circ2_isa], runs=2048)
dists = result.data
```
d.
```python
from qiskit_ibm_runtime import SamplerV2
sampler = SamplerV2(mode=backend)
job = sampler.run([circ1, circ2])  # QuantumCircuit objects
dists = job.result()
```


:::{dropdown}answer 
The answer is a. 

`SamplerV2.run` accepts a list of ISA circuits and returns a job whose `result()` contains per‑circuit distribution payloads. Passing multiple positional circuit arguments or non‑ISA `QuantumCircuit` objects is not correct for the runtime API. 
:::

25. What is `options.seed_estimator` used for in EstimatorV2?

a. To control sampling randomness (e.g., shot sampling and randomizations) via a fixed seed.<br>
b. To seed the transpiler’s layout selection.<br>
c. To fix the hardware qubit mapping across sessions.<br>
d. To seed the optimizer used in VQE.<br>


:::{dropdown}answer 
The answer is a. 

`seed_estimator` sets a seed to control stochastic parts of the estimator’s execution, such as sampling and randomized procedures, improving reproducibility across runs.
:::


## TASK 5.2/6.2: Understand the theoretical background behind the sampler/estimator primitive

1. Which snippet best demonstrates **multiple PUBs** in one call to Sampler V2?

a.
```python
pubs = [(qc1, [theta1], 2000), (qc2, [theta2], 4000)]
job = sampler.run(pubs)
res = job.result()
```
b.
```python
pubs = [(qc1, obs1, [theta1]), (qc2, obs2, [theta2])]
job = sampler.run(pubs)
```
c.
```python
pubs = [([qc1, qc2], {'shots': [2000, 4000]})]
job = sampler.run(pubs)
```
d.
```python
pubs = {qc1: 2000, qc2: 4000}
job = sampler.run(pubs)
```

:::{dropdown}answer 
The answer is a. 

A Sampler PUB can be expressed as `(circuit, <optional> parameter_values, <optional> shots)`. Multiple PUBs can be submitted in a single `run` call.
:::


2. Which of the following best describes the **output** of the Sampler primitive?

a. A histogram of expectation values for given observables<br>
b. A quasi-probability distribution over bitstrings measured from the circuit<br>
c. A sequence of transpiled circuits optimized for the backend<br>
d. A set of error-mitigated observables in operator form<br>

:::{dropdown}answer
The answer is b.

The Sampler outputs a **quasi-probability distribution** that summarizes how often each bitstring is expected to occur. This distribution captures sampling statistics, while expectation values belong to Estimator results.
:::

3. You need to obtain **shot-by-shot samples** from a circuit while preserving the measurement order for dynamic-circuit compatibility. Which method should you use?

a. Use `EstimatorV2.run([qc])` to return expectation values per shot.<br>
b. Use `SamplerV1.run([qc])` to return a quasi-probability distribution.<br>
c. Use `SamplerV2.run([qc])` to return samples of classical outcomes, preserving shot order.<br>
d. Use `QuantumCircuit.sample_counts(qc)` to return per-shot bit arrays.

:::{dropdown}answer 
The answer is c. 

`SamplerV2` focuses on sampling the output register and returns **samples of classical outcomes**, preserving the **shot order**. This contrasts with Sampler V1, which returns quasi-probability distributions. Estimator computes expectation values, not samples.
:::

4. You want to submit **several circuits, each with its own parameter values and (optionally) shots**, in a single call. Which interface best matches the Sampler V2 design?

a. Provide a list of PUBs, where each PUB is `(circuit, <optional> parameter_values, <optional> shots>)`.<br>
b. Provide a list of `(observable, circuit)` pairs.<br>
c. Provide a dictionary mapping circuits to quasi-probabilities.<br>
d. Provide a single circuit with a batch size argument `batch=True`.

:::{dropdown}answer 
The answer is a. 

Sampler V2 accepts multiple **Primitive Unified Blocs (PUBs)**, each tuple typically `(circuit, <optional> parameter_values, <optional> shots)`, so you can submit many circuit evaluations together.
:::


5. In Sampler V2, results are organized by **classical register names** from the circuit. Which description best matches this behavior?

a. Samples are returned as raw bytes, with no register structure available.<br>
b. Samples are grouped **per classical register**, aiding compatibility with dynamic circuits.<br>
c. Samples are flattened into integers and sorted by frequency.<br>
d. Samples are returned only as aggregate counts per register.

:::{dropdown}answer 
The answer is b. 

Sampler V2 organizes output by the **output/classical register names** defined by the input program, facilitating use with dynamic circuits.
:::


6. In the context of the Estimator primitive, what is the main theoretical effect of **Pauli twirling** on gate errors?

a. It only applies to single-qubit gates and leaves two-qubit noise unchanged.<br>
b. It eliminates measurement (readout) errors by repeating measurements with majority voting.<br>
c. It exactly cancels hardware noise by inverting it gate-by-gate.<br>
d. It transforms arbitrary noise into a Pauli channel so that coherent errors accumulate linearly rather than quadratically.<br>

:::{dropdown}answer 
The answer is d. 

Pauli twirling (randomized compiling with Pauli gates) maps general noise to a Pauli channel. This changes error growth from quadratic (coherent) to linear (Pauli), improving expectation-value estimation by the Estimator when combined with sampling over randomized circuit instances. 
:::


7. You estimate expectation values of Pauli observables and see bias dominated by readout noise. Which technique directly targets *measurement* errors for Estimator?

a. Pauli twirling (gate twirling)<br>
b. Twirled Readout Error eXtinction (TREX)<br>
c. Dynamical decoupling (DD)<br>
d. Zero-noise extrapolation (ZNE)<br>

:::{dropdown}answer 
The answer is b. 

TREX performs measurement twirling (random X before measure + classical bit flip) to diagonalize the readout-error transfer matrix and remove readout bias from expectation values. Gate twirling and DD address gate/coherent errors; ZNE mitigates circuit noise via noise scaling and extrapolation, not specifically readout. 
:::


3. **Zero-noise extrapolation (ZNE)**, as used with the Estimator, is best summarized by which two-stage process?

a. Learn a quasi-probability model and sample from it to reproduce the ideal circuit exactly.<br>
b. Amplify noise (e.g., by digital gate folding) and then extrapolate expectation values to the zero-noise limit.<br>
c. Insert idle-pulse sequences to cancel coherent errors during “gaps” in the schedule.<br>
d. Calibrate a confusion matrix and invert it to de-bias measurement outcomes.<br>

:::{dropdown}answer 
The answer is b. 

ZNE runs the same logical circuit at different effective noise levels (e.g., via gate folding like replacing \(U\) with \(UU^\dagger U\)), then fits an extrapolator (linear, exponential, etc.) to estimate the zero-noise expectation value.
:::


4. In **digital gate folding** for ZNE, replacing a unitary $U$ by $UU^{\dagger}U$ corresponds to which **noise amplification factor**?

a. 3<br>
b. 2<br>
c. 1<br>
d. 0<br>

:::{dropdown}answer 
The answer is a. 

Qiskit Runtime implements noise amplification by "digital gate folding," which means that two-qubit gates are replaced with equivalent sequences of the gate and its inverse.

The sequence $UU^{\dagger}U$ increases the effective number of noisy operations by a factor (`noise_factors`) of 3 while preserving the ideal unitary action, enabling controlled noise amplification for extrapolation.

- noise_factors: The noise factors to use for noise amplification.
:::


5. **Probabilistic Error Amplification (PEA)** provides a more accurate noise amplification than simple folding. Which statement captures its core idea?

a. It learns a twirled noise model for each entangling layer and then injects single-qubit noise proportionally during amplification.<br>
b. It only repeats two-qubit gates and their inverses to scale noise without any learning.<br>
c. It yields an exactly unbiased estimator of the expectation value without overhead.<br>
d. It does not require ZNE to be enabled because it is a suppression (not mitigation) method.<br>

:::{dropdown}answer 
The answer is a. 

PEA first **learns** the twirled noise per entangling layer, then **amplifies** by probabilistically injecting single‑qubit noise consistent with that learned model before **extrapolating** to the zero‑noise limit. It is typically used alongside ZNE in Estimator workflows.
:::


6. You need an *unbiased* estimator of ⟨O⟩ but can tolerate the highest shot overhead. Which mitigation method matches this requirement?

a. ZNE with linear extrapolation<br>
b. PEC (probabilistic error cancellation)<br>
c. TREX with many randomizations<br>
d. Gate twirling only<br>

:::{dropdown}answer 
The answer is b. 

PEC reconstructs the ideal channel as a quasi-probability mixture of noisy channels, yielding an *unbiased* expectation value estimate at the cost of large sampling overhead that scales with the quasi-probability L1 norm. ZNE often reduces bias but is not guaranteed unbiased; TREX and gate twirling address different error sources. 
:::


7. In **Pauli gate twirling** as implemented for Estimator runs, how are **randomized circuit instances** and shots typically managed?

a. They change the backend’s native basis gates to Pauli-only operations.<br>
b. Randomizations only affect calibration circuits and never the main circuit instances.<br>
c. The parameters set the ZNE noise factors and the extrapolator form.<br>
d. The Estimator samples multiple randomized instances (controlled by `num_randomizations`) and takes `shots_per_randomization` shots from each.<br>

:::{dropdown}answer 
The answer is d. 

Gate twirling replaces a single circuit with an ensemble of randomized, Pauli‑wrapped instances that have the same ideal effect. The Estimator then draws a specified number of instances and shots per instance to estimate expectation values.
:::


8. For a Hermitian observable and real-valued expectation ⟨O⟩, what is the expected *data type* of the Estimator’s returned value (ignoring confidence intervals and metadata)?

a. A dictionary mapping bitstrings to counts<br>
b. A complex number with generally nonzero imaginary part<br>
c. A real scalar (float) expectation value<br>
d. A list of complex amplitudes (statevector)<br>

:::{dropdown}answer 
The answer is c. 

The Estimator primitive computes expectation values of observables; for Hermitian observables, the expectation is real and is returned as a real scalar. Counts dictionaries and statevectors belong to different primitives/tools; a nonzero imaginary part would contradict Hermiticity. 
:::


9. Conceptually, how do **error suppression** and **error mitigation** differ for Estimator‑based workflows?

a. Suppression modifies the implemented circuit/pulse schedule to reduce noise during execution (e.g., DD, gate twirling), whereas mitigation post‑processes or re‑processes experimental data to adjust expectation values (e.g., TREX, ZNE, PEC).<br>
b. Both terms mean the same thing and are interchangeable in practice.<br>
c. Mitigation changes gate calibrations in hardware; suppression extrapolates to the zero‑noise limit.<br>
d. Suppression is only about measurement, while mitigation is only about two‑qubit gates.<br>

:::{dropdown}answer 
The answer is a. 

Suppression techniques alter what is executed (added pulses, randomized wrappers) to reduce error accumulation, while mitigation techniques change how results are **estimated** (diagonalizing readout noise, extrapolating to zero noise, or canceling errors via quasi‑probabilities) to recover better expectation values.
:::


10. Which Estimator option enables *measurement* error mitigation via TREX according to the documented resilience settings?

a. `estimator.options.twirling.enable_gates = True`<br>
b. `estimator.options.resilience.measure_mitigation = True`<br>
c. `estimator.options.dynamical_decoupling.enable = True`<br>
d. `estimator.options.resilience.zne_mitigation = True`<br>


:::{dropdown}answer 
The answer is b. 

`resilience.measure_mitigation` turns on TREX-based readout mitigation for Estimator. Gate twirling and DD are different options; `zne_mitigation` enables ZNE, not measurement mitigation. 
:::


11. You want ZNE with specific amplification factors and an exponential extrapolator. Which option pair matches the documented keys?

a. `estimator.options.zne.noise_factors`, `estimator.options.zne.extrapolator`<br>
b. `estimator.options.resilience.zne.noise_factors`, `estimator.options.resilience.zne.extrapolator`<br>
c. `estimator.options.zne.factors`, `estimator.options.zne.method='exponential'`<br>
d. `estimator.options.resilience.zne.factors`, `estimator.options.resilience.zne.fit='exp'`<br>


:::{dropdown}answer 
The answer is b. 

ZNE is configured under `options.resilience.zne`, with keys like `noise_factors` and `extrapolator` (e.g., `(1, 3, 5)` and `"exponential"`). Other key paths/names are not documented. 
:::
